In [30]:
import pandas as pd
df = pd.read_excel('Doku Langenbach 29-04-24.xlsx', sheet_name='LT22 Export', header=0)

In [31]:
from Data_Class.MMSQL_connection import read_Table, save_Table
# filter df by Queue GITPD 
dfGITPD = df[df['Queue'] == 'GITPD']
dfMasterdata = read_Table('data_materialmaster-MaterialMasterUnitOfMeasures')
dfPAMSMasterdata = read_Table('PAMS_TALL_MASTERDATA')



In [32]:
# def cal_Order_Qty_and_MasterData_SAVE_TO_DATABASE(dfMasterdata: pd.DataFrame, df: pd.DataFrame):
#     '''This function calculates the Order Quantity for Case and PAL and merges the Masterdata to the df
#     :return: save to SQL Table = Tall_Pall_Orders'''

dfMasterdata['MaterialNumber'] = dfMasterdata['MaterialNumber'].str.replace('0000000000', '')

df['Material'] = df['Material'].astype(str)
df_merged_Case = pd.merge(df, dfMasterdata[dfMasterdata['UnitOfMeasure'] == 'CS'], left_on='Material', right_on='MaterialNumber', how='left')
df_merged_Case['MM_TH_Pro_Karton'] = df_merged_Case.NumeratorToBaseUnitOfMeasure / df_merged_Case.DenominatorToBaseUnitOfMeasure
df['MM_TH_Pro_Karton'] = df_merged_Case['MM_TH_Pro_Karton']
df['height_Case'] = df_merged_Case['Height']
df['length_Case'] = df_merged_Case['Length']
df['width_Case'] = df_merged_Case['Width']

df_merged_pal = pd.merge(df, dfMasterdata[dfMasterdata['UnitOfMeasure'] == 'D97'], left_on='Material', right_on='MaterialNumber', how='left')
df_merged_pal['MM_TH_Pro_Palette'] = df_merged_pal.NumeratorToBaseUnitOfMeasure / df_merged_pal.DenominatorToBaseUnitOfMeasure
df['MM_TH_Pro_Palette'] = df_merged_pal['MM_TH_Pro_Palette']
df['height_PAL'] = df_merged_pal['Height']
df['length_PAL'] = df_merged_pal['Length']
df['width_PAL'] = df_merged_pal['Width']

df_merged_PD = pd.merge(df, dfMasterdata[dfMasterdata['UnitOfMeasure'] == 'PD'], left_on='Material', right_on='MaterialNumber', how='left')
df_merged_PD['Case_on_Layer'] = df_merged_PD.NumeratorToBaseUnitOfMeasure / df_merged_PD.DenominatorToBaseUnitOfMeasure
df['Case_on_Layer'] = df_merged_PD['Case_on_Layer']

# ########## UNTERBAU  ##########
dfPAMSMasterdata['SKU'] = dfPAMSMasterdata['SKU'].astype(str)
df = pd.merge(df, dfPAMSMasterdata, left_on='Material', right_on='SKU', how='left')
nan_indices = df[df['AUFPACKEN_JA_NEIN'].isna()].index
df.loc[nan_indices, 'GEEIGNET_UNTERBAU'] = 'FALSE'
df.loc[nan_indices, 'AUFPACKEN_JA_NEIN'] = 'FALSE'



In [33]:
import uuid
df['Teilbar_durch'] =  df['MM_TH_Pro_Palette'].astype(int) / df['Case_on_Layer'].astype(int)
# #drop MM_Case_per_PAL and Case_on_Layer
df['Teilbar_durch'] = df['Teilbar_durch'].astype(float)
# # df = df.drop(columns=['MM_Case_per_PAL', 'Case_on_Layer'])
# # set unique ID for each row 
df['Pal_ID'] = [uuid.uuid4() for _ in range(len(df.index))]
df['Teilbar_durch'] = df['Teilbar_durch'].apply(lambda x: round(x))
df['Teilhöhe'] = df['height_PAL'].astype(float) / df['Teilbar_durch']
df['Teilhöhe'] = df['Teilhöhe'].apply(lambda x: round(x, 2)) + 1


In [34]:
df = df[df['Queue'] == 'GITPD']
df = pd.read_csv('dfPal.csv')
df_pal = df.copy()
df

In [35]:
# Festlegen der maximalen Höhe
max_height = 270

# Erstelle eine Kopie des DataFrames, um die Bearbeitung und Tracking zu erleichtern
df_stacking = df_pal.copy()

# Hinzufügen einer Spalte zur Verfolgung der verbleibenden Höhe, die hinzugefügt werden kann
df_stacking['verbleibende_Höhe'] = max_height - df_stacking['height_PAL']

# Sortiere die Paletten nach geeignetem Unterbau und dann nach der Möglichkeit aufzupacken
df_stacking.sort_values(by=['GEEIGNET_UNTERBAU', 'AUFPACKEN_JA_NEIN', 'height_PAL'], ascending=[False, False, True], inplace=True)

# Liste zur Erfassung der Stapelungsdetails
stapel_details = []

# Durchlaufe die Paletten, um Stapelungen zu planen
for index, target in df_stacking[df_stacking['GEEIGNET_UNTERBAU']].iterrows():
    if target['verbleibende_Höhe'] <= 0:
        continue
    
    # Identifiziere mögliche Paletten, die auf diese Palette gestapelt werden könnten
    for idx, source in df_stacking.iterrows():
        if source['AUFPACKEN_JA_NEIN'] and source['Pal_ID'] != target['Pal_ID']:
            maximale_teile = int(target['verbleibende_Höhe'] / source['Teilhöhe'])
            stapelbare_teile = min(source['Teilbar_durch'], maximale_teile)
            if stapelbare_teile > 0:
                neue_höhe = target['height_PAL'] + stapelbare_teile * source['Teilhöhe']
                stapel_details.append({
                    'Quell_Pal_ID': source['Pal_ID'],
                    'Ziel_Pal_ID': target['Pal_ID'],
                    'Anzahl_Teile': stapelbare_teile,
                    'Teilhöhe': source['Teilhöhe'],
                    'Ausgangshöhe': target['height_PAL'],
                    'Neue_Höhe': neue_höhe
                })
                # Update der Höhen in den DataFrames
                df_stacking.at[index, 'height_PAL'] = neue_höhe
                df_stacking.at[index, 'verbleibende_Höhe'] = max_height - neue_höhe
                df_stacking.at[idx, 'Teilbar_durch'] -= stapelbare_teile

                if df_stacking.at[idx, 'Teilbar_durch'] <= 0:
                    df_stacking.drop(idx, inplace=True)
                break  # Breche die innere Schleife ab, wenn wir gestapelt haben

# Filtere unverbrauchte Paletten
unverbrauchte_paletten = df_stacking[df_stacking['Teilbar_durch'] > 0]

stapel_details, unverbrauchte_paletten[['Pal_ID', 'Teilbar_durch', 'height_PAL']]


([{'Quell_Pal_ID': '2caeb854-ee96-46a3-80fa-32d453ba9eca',
   'Ziel_Pal_ID': 'dff41ed0-13f2-49c9-871b-78d110b42374',
   'Anzahl_Teile': 4,
   'Teilhöhe': 21.33,
   'Ausgangshöhe': 166.5,
   'Neue_Höhe': 251.82},
  {'Quell_Pal_ID': 'dff41ed0-13f2-49c9-871b-78d110b42374',
   'Ziel_Pal_ID': '2caeb854-ee96-46a3-80fa-32d453ba9eca',
   'Anzahl_Teile': 1,
   'Teilhöhe': 56.5,
   'Ausgangshöhe': 183.0,
   'Neue_Höhe': 239.5},
  {'Quell_Pal_ID': 'dff41ed0-13f2-49c9-871b-78d110b42374',
   'Ziel_Pal_ID': 'ca8ff28d-e679-4433-b4f0-37f42aebc453',
   'Anzahl_Teile': 1,
   'Teilhöhe': 56.5,
   'Ausgangshöhe': 183.0,
   'Neue_Höhe': 239.5},
  {'Quell_Pal_ID': 'dff41ed0-13f2-49c9-871b-78d110b42374',
   'Ziel_Pal_ID': 'd3212cb1-8efb-4e26-b191-36c5f2630f03',
   'Anzahl_Teile': 1,
   'Teilhöhe': 56.5,
   'Ausgangshöhe': 183.0,
   'Neue_Höhe': 239.5},
  {'Quell_Pal_ID': '2caeb854-ee96-46a3-80fa-32d453ba9eca',
   'Ziel_Pal_ID': '2aeb1179-f411-49be-b9bc-b887cad67ddf',
   'Anzahl_Teile': 4,
   'Teilhöhe': 21.3

In [36]:
import pandas as pd

# Annahme: DataFrame 'df' ist bereits definiert
# df = pd.read_csv('path_to_your_file.csv')  # So könntest du deine Daten laden

class Palette:
    def __init__(self, id, teilbar_durch, teilhoehe, height_pal, aufpacken, geeignet_unterbau):
        self.id = id
        self.teilbar_durch = teilbar_durch
        self.teilhoehe = teilhoehe
        self.height_pal = height_pal
        self.aufpacken = aufpacken
        self.geeignet_unterbau = geeignet_unterbau

# Umwandlung der DataFrame-Zeilen in Palette-Objekte
paletten = [Palette(row['Pal_ID'], row['Teilbar_durch'], row['Teilhöhe'], row['height_PAL'], row['AUFPACKEN_JA_NEIN'], row['GEEIGNET_UNTERBAU']) for index, row in df.iterrows()]

def optimiere_paletten(paletten):
    ergebnisse = []
    unverbrauchte_paletten = paletten.copy()
    unterbau_paletten = [p for p in paletten if p.geeignet_unterbau]
    aufteilbare_paletten = [p for p in paletten if p.aufpacken]

    for p in aufteilbare_paletten:
        teile_zu_stapeln = p.teilbar_durch
        teilhoehe = p.teilhoehe
        while teile_zu_stapeln > 0:
            verbraucht = False
            for u in unterbau_paletten:
                if u.height_pal + teilhoehe <= 270:
                    max_teile = int((270 - u.height_pal) / teilhoehe)
                    stapel_teile = min(teile_zu_stapeln, max_teile)
                    u.height_pal += stapel_teile * teilhoehe
                    ergebnisse.append(f"{stapel_teile} Teile von {p.id} auf {u.id} gestapelt. Neue Höhe: {u.height_pal}")
                    teile_zu_stapeln -= stapel_teile
                    if teile_zu_stapeln == 0:
                        verbraucht = True
                        break
            if not verbraucht:
                break

    # Paletten, die nicht verwendet wurden, auflisten
    for p in paletten:
        if p in unverbrauchte_paletten:
            ergebnisse.append(f"Palette {p.id} wurde nicht verwendet und bleibt unversehrt")

    return ergebnisse

# Ergebnisse ausführen und drucken
ergebnisse = optimiere_paletten(paletten)
for ergebnis in ergebnisse:
    print(ergebnis)


1 Teile von dff41ed0-13f2-49c9-871b-78d110b42374 auf dff41ed0-13f2-49c9-871b-78d110b42374 gestapelt. Neue Höhe: 223.0
1 Teile von dff41ed0-13f2-49c9-871b-78d110b42374 auf 2caeb854-ee96-46a3-80fa-32d453ba9eca gestapelt. Neue Höhe: 239.5
1 Teile von dff41ed0-13f2-49c9-871b-78d110b42374 auf ca8ff28d-e679-4433-b4f0-37f42aebc453 gestapelt. Neue Höhe: 239.5
1 Teile von 6f2d3a4d-38b5-47de-9d13-bea80c094878 auf d3212cb1-8efb-4e26-b191-36c5f2630f03 gestapelt. Neue Höhe: 245.77
1 Teile von 6f2d3a4d-38b5-47de-9d13-bea80c094878 auf 2aeb1179-f411-49be-b9bc-b887cad67ddf gestapelt. Neue Höhe: 245.77
1 Teile von 6f2d3a4d-38b5-47de-9d13-bea80c094878 auf d3176172-61e8-4fb3-95c2-aa8edfb19aac gestapelt. Neue Höhe: 245.77
1 Teile von ca92e215-dc02-4632-bbfd-4cdf23bf0f6d auf 3ad13741-77de-45f5-a59f-957e5ba7d92f gestapelt. Neue Höhe: 263.17
1 Teile von ca92e215-dc02-4632-bbfd-4cdf23bf0f6d auf 83b9f797-c7a7-450f-be79-b90556fb94b5 gestapelt. Neue Höhe: 263.17
1 Teile von ca92e215-dc02-4632-bbfd-4cdf23bf0f6d au

In [37]:
import pandas as pd

def optimize_pallet_stacking(df, max_height=270):
    # Definition der Ergebnisliste
    optimized_results = []

    # Filtern der Paletten, die als Unterbau dienen können
    base_palettes = df[df['GEEIGNET_UNTERBAU'] & (df['height_PAL'] < max_height)]

    # Durchlauf durch jede Palette, die als Unterbau verwendet werden kann
    for base_id in base_palettes['Pal_ID'].unique():
        base_palette = df[df['Pal_ID'] == base_id]
        current_height = base_palette['height_PAL'].values[0]
        stackable_palettes = df[(df['AUFPACKEN_JA_NEIN']) & (df['Pal_ID'] != base_id)]

        for _, stackable in stackable_palettes.iterrows():
            part_height = stackable['Teilhöhe']
            max_parts = (max_height - current_height) // part_height
            parts_to_add = min(max_parts, stackable['Teilbar_durch'])

            if parts_to_add > 0:
                current_height += parts_to_add * part_height
                optimized_results.append({
                    'Base_ID': base_id,
                    'Stack_ID': stackable['Pal_ID'],
                    'Parts_Added': parts_to_add,
                    'Final_Height': current_height,
                })

                if current_height >= max_height:
                    break

    # Erstellen eines DataFrame aus den optimierten Ergebnissen
    optimized_df = pd.DataFrame(optimized_results)
    optimized_summary_df = optimized_df.groupby(['Base_ID', 'Stack_ID']).agg(
        Total_Parts_Added=('Parts_Added', 'sum'),
        Max_Final_Height=('Final_Height', 'max')
    ).reset_index().sort_values(by='Max_Final_Height', ascending=False)

    # Umbenennen der Spalten für die Ausgabe
    optimized_summary_df.rename(columns={
        'Base_ID': 'Unterbau-Palette ID',
        'Stack_ID': 'Gestapelte Palette ID',
        'Total_Parts_Added': 'Gesamtanzahl gestapelter Teile',
        'Max_Final_Height': 'Maximale Endhöhe'
    }, inplace=True)

    return optimized_summary_df

# Beispiel zur Verwendung der Funktion:
# df = pd.read_csv('dein_palletten_csv_pfad.csv')
result_df = optimize_pallet_stacking(df)
print(result_df)


                     Unterbau-Palette ID  \
24  7e569dd8-02b3-48c3-979a-58ad5feab89b   
7   277a548d-68bd-4bbb-95c8-642c27ab5186   
15  35a77a5e-87e3-422a-ba5c-e6c4306ccec4   
40  d83be558-3d46-47cf-b15b-8389c4966a27   
47  fbc9be12-1157-4826-ad1d-5e1aa6a4811f   
36  d3176172-61e8-4fb3-95c2-aa8edfb19aac   
38  d3212cb1-8efb-4e26-b191-36c5f2630f03   
33  ca8ff28d-e679-4433-b4f0-37f42aebc453   
10  2aeb1179-f411-49be-b9bc-b887cad67ddf   
12  2caeb854-ee96-46a3-80fa-32d453ba9eca   
32  ca024741-a424-4526-b10e-073c0492b544   
28  846c8610-e5c5-47d9-8ec6-836c14c2f968   
27  83b9f797-c7a7-450f-be79-b90556fb94b5   
21  5b0b27f9-c2d4-41c6-9061-9b020fc4218e   
20  4d4e9114-e16a-478d-aec9-397628e950d4   
19  3ad13741-77de-45f5-a59f-957e5ba7d92f   
18  38c94057-9eeb-4168-8129-d931f85a2085   
43  d879a8b9-9a58-4a44-bb42-98c7a76c4690   
3   1f5f49c2-d50d-45ae-a46f-040a8e78ae31   
9   2aaf2ef2-a96e-43cb-a341-7b2cbdf0b40c   
17  3857e7c7-dfd9-4ecf-bbec-12f776ea8536   
6   2534475f-39e6-4c3c-adfe-4fb5

In [38]:
# Maximale Höhe einer Palette in cm
MAX_HEIGHT = 270
df_pal = df.copy()
# Ergebnis DataFrame vorbereiten
results = []

# Paletten auswählen, die als Unterbau dienen können
base_palettes = df_pal[df_pal['GEEIGNET_UNTERBAU'] & (df_pal['height_PAL'] < MAX_HEIGHT)]

# Über alle Unterbau-Paletten iterieren
for index, base in base_palettes.iterrows():
    current_height = base['height_PAL']
    base_id = base['Pal_ID']
    
    # Paletten finden, deren Teile auf den Unterbau gestapelt werden können
    stackable_palettes = df_pal[df_pal['AUFPACKEN_JA_NEIN'] & (df_pal['Pal_ID'] != base_id)]
    
    # Stapeln der Teile auf den Unterbau
    for _, stackable in stackable_palettes.iterrows():
        stack_id = stackable['Pal_ID']
        part_height = stackable['Teilhöhe']
        
        # Berechnen, wie viele Teile maximal gestapelt werden können
        max_parts = (MAX_HEIGHT - current_height) // part_height
        parts_to_add = min(max_parts, stackable['Teilbar_durch'])
        
        if parts_to_add > 0:
            # Update der aktuellen Höhe des Unterbaus
            current_height += parts_to_add * part_height
            # Ergebnis speichern
            results.append({
                'Base_ID': base_id,
                'Stack_ID': stack_id,
                'Parts_Added': parts_to_add,
                'Final_Height': current_height,
                'Parts_Height': parts_to_add * part_height
            })
            
        # Stoppe das Stapeln, wenn die Maximalhöhe erreicht ist
        if current_height >= MAX_HEIGHT:
            break
    
    # Update der Daten, um die verwendeten Teile zu markieren
    df_pal.loc[df_pal['Pal_ID'] == stack_id, 'Teilbar_durch'] -= parts_to_add

# Ergebnisse als DataFrame konvertieren
result_df = pd.DataFrame(results)
result_df


,Base_ID,Stack_ID,Parts_Added,Final_Height,Parts_Height
0,dff41ed0-13f2-49c9-871b-78d110b42374,6f2d3a4d-38b5-47de-9d13-bea80c094878,1.0,229.27,62.77
1,dff41ed0-13f2-49c9-871b-78d110b42374,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,250.60,21.33
2,2caeb854-ee96-46a3-80fa-32d453ba9eca,dff41ed0-13f2-49c9-871b-78d110b42374,1.0,239.50,56.50
3,2caeb854-ee96-46a3-80fa-32d453ba9eca,ca8ff28d-e679-4433-b4f0-37f42aebc453,1.0,260.83,21.33
4,ca8ff28d-e679-4433-b4f0-37f42aebc453,dff41ed0-13f2-49c9-871b-78d110b42374,1.0,239.50,56.50
5,ca8ff28d-e679-4433-b4f0-37f42aebc453,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,260.83,21.33
6,d3212cb1-8efb-4e26-b191-36c5f2630f03,dff41ed0-13f2-49c9-871b-78d110b42374,1.0,239.50,56.50
7,d3212cb1-8efb-4e26-b191-36c5f2630f03,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,260.83,21.33
8,2aeb1179-f411-49be-b9bc-b887cad67ddf,dff41ed0-13f2-49c9-871b-78d110b42374,1.0,239.50,56.50
9,2aeb1179-f411-49be-b9bc-b887cad67ddf,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,260.83,21.33


In [39]:
# Zusammenfassen der Daten für effizientere Stapelvorgänge
# Gruppieren nach Base_ID und Stack_ID, Summierung der gestapelten Teile und Maximalwert der Höhe nehmen
summary_df = result_df.groupby(['Base_ID', 'Stack_ID']).agg(
    Total_Parts_Added=('Parts_Added', 'sum'),
    Max_Final_Height=('Final_Height', 'max')
).reset_index()

# Sortieren der Ergebnisse für effektivere Darstellung
sorted_summary_df = summary_df.sort_values(by='Max_Final_Height', ascending=False)

# Ergebnisse bereinigen für die Endausgabe
final_output = sorted_summary_df.rename(columns={
    'Base_ID': 'Unterbau-Palette ID',
    'Stack_ID': 'Gestapelte Palette ID',
    'Total_Parts_Added': 'Gesamtanzahl gestapelter Teile',
    'Max_Final_Height': 'Maximale Endhöhe'
})

# Ausgabe der finalen, zusammengefassten Ergebnisse
final_output


,Unterbau-Palette ID,Gestapelte Palette ID,Gesamtanzahl gestapelter Teile,Maximale Endhöhe
24,7e569dd8-02b3-48c3-979a-58ad5feab89b,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,265.83
7,277a548d-68bd-4bbb-95c8-642c27ab5186,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,265.83
15,35a77a5e-87e3-422a-ba5c-e6c4306ccec4,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,265.83
40,d83be558-3d46-47cf-b15b-8389c4966a27,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,263.33
47,fbc9be12-1157-4826-ad1d-5e1aa6a4811f,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,263.13
36,d3176172-61e8-4fb3-95c2-aa8edfb19aac,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,260.83
38,d3212cb1-8efb-4e26-b191-36c5f2630f03,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,260.83
33,ca8ff28d-e679-4433-b4f0-37f42aebc453,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,260.83
10,2aeb1179-f411-49be-b9bc-b887cad67ddf,2caeb854-ee96-46a3-80fa-32d453ba9eca,1.0,260.83
12,2caeb854-ee96-46a3-80fa-32d453ba9eca,ca8ff28d-e679-4433-b4f0-37f42aebc453,1.0,260.83
